<a href="https://colab.research.google.com/github/kyoungmin-park/univ-starter/blob/main/%EB%8C%80%ED%95%99%EB%B6%80%EC%8A%A4%ED%83%80%ED%84%B02_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import json
import pandas as pd
from collections import defaultdict

# 라벨링 데이터 경로 설정
train_label_dir = '/content/drive/Othercomputers/내 노트북/010.연도별 패션 선호도 파악 및 추천 데이터/01-1.정식개방데이터/Training/02.라벨링데이터'
val_label_dir = '/content/drive/Othercomputers/내 노트북/010.연도별 패션 선호도 파악 및 추천 데이터/01-1.정식개방데이터/Validation/02.라벨링데이터'

def parse_labels(label_dir):
    """주어진 폴더 내 JSON 파일을 읽어 respondent_id별 선호/비선호 스타일 개수를 반환"""
    pref_count = defaultdict(int)
    nonpref_count = defaultdict(int)

    for filename in os.listdir(label_dir):
        if filename.endswith('.json'):
            parts = filename.replace('.json', '').split('_')
            if len(parts) == 6:
                json_path = os.path.join(label_dir, filename)
                try:
                    with open(json_path, 'r', encoding='utf-8') as f:
                        data = json.load(f)
                        rid = data['user']['R_id']
                        q5 = data['item']['survey'].get('Q5', [])
                        q6 = data['item']['survey'].get('Q6', [])

                        if isinstance(q5, list):
                            pref_count[rid] += len(q5)
                        if isinstance(q6, list):
                            nonpref_count[rid] += len(q6)
                except Exception:
                    continue
    return pref_count, nonpref_count

# 1. 먼저 training 데이터로부터 top 100명 추출
train_pref, _ = parse_labels(train_label_dir)
top_100_ids = sorted(train_pref.keys())[:100]  # 알파벳순 상위 100명 기준

# 2. training/validation 데이터 모두 처리
train_pref, train_nonpref = parse_labels(train_label_dir)
val_pref, val_nonpref = parse_labels(val_label_dir)

# 3. 표 구성
data = {
    'train_preference': [train_pref[rid] for rid in top_100_ids],
    'train_non_preference': [train_nonpref[rid] for rid in top_100_ids],
    'val_preference': [val_pref[rid] for rid in top_100_ids],
    'val_non_preference': [val_nonpref[rid] for rid in top_100_ids],
}

# 4. 데이터프레임으로 변환 (행-열 전치)
df = pd.DataFrame(data, index=top_100_ids).T  # transpose 해서 행이 조건, 열이 ID
print(df)
